Github repo: https://github.com/jprieto42/CS598DL4H_Project

Video link : https://youtu.be/jR8m8GHRGF0

# Introduction
In Proteomics one of the challenges in proteomic analysis is the stochastic nature of peptide selection in mass spectrometry (MS). Peptide detectability, which is the ability of a peptide to be accurately detected by MS, is necessary to identify the protein and quantify it. Existing methods often overlook peptide detectability, leading to inaccurate results and hindered progress in proteomic research. The paper presents DeepMSPeptide, a bioinformatic tool that uses a deep learning method to predict which peptides can be detected in MS exclusively based on the peptide amino acid sequences.

The specific approach of DeepMSPeptide utilizes a convolutional neural network (CNN) architecture to predict the detectability of peptides based solely on their amino acid sequences. The input vector of the DeepMSPeptide classifier is created using a CNN in TensorFlow by assigning an integer to each amino acid. The input is standardized for the CNN by performing zero-padding because of the different lengths of the peptides which results in each vector with a size of 81 elements. The first layer of the model is an embedding layer. The second layer is a dropout layer to prevent over-fitting. The next layers include 1 or 2 1D convolutional layers applying 64 filters with a sliding window of 1 and a width of 3. Lastly, the model includes two additional hidden layers. Those layers include a single-unit layer with a sigmoid activation function and the loss function selected for training was the binary cross entropy. Each model is trained for 200 epochs in batches of 100 samples. The accuracy is measured during the training with a fourth of the samples as the validation set and a call-back function to stop the training when the loss function is stabilized during five consecutive epochs. The input data used to train and test the different classifier approaches were obtained from the GPMDB database. This papers contribution was specifically adding CNNs to better learn the relationships of the amino acid sequences before passing them into the deep learning model. Previously the best performing model was that of random forrests but this papers model was able to surpass them. The importance of this contribution is that they have furthered the performance in the ability to predict petide detectability using deep learning methods which will ultimately help researchers make more discoveries in cell biology and human disease.


# Scope of Reproducibility:

The paper does not explicitly list the hypothesis being tested but these are implied by its methods being tested.

Hypothesis 1: The CNN-DL will outperform previous existing machine learning (ML) models.
- To test this I will compare performance statistics of the models against performance statistics of previous ML models.

# Methodology

## Data

- Source of the data: The input data used to train and test the different classifier approaches were obtained from the GPMDB database, which includes thousands of peptides detected in hundreds of MS/MS experiments and the frequency of detection of each of them.[1] The data used to build the model is not needed for the testing of the model because it is pre trained and compiled.
- Statistics: The statisctics for the train and test datasets are includede below.The split to train the model was 75% train 25 % test.
- Data process: The data must be processed first by reading the first and last columns of the GPMDB dataset that correspond to the peptide sequence and class respectively. The class consist of two classes, MObs which is a peptide that is most observed by MS and LObs which is a peptide that is least observed by MS. The classes are then converted to binary to match adapted code output, MObs = 1 and LObs = 0.



In [41]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import csv
from sklearn import metrics

In [42]:
# dir and function to load raw data

def load_pep_and_codify(file, max_len):
    aa_dict={'A':1,'R':2,'N':3,'D':4,'C':5,'Q':6,'E':7,'G':8,'H':9,'I':10,'L':11,'K':12,'M':13,'F':14,
        'P':15,'O':16,'S':17,'U':18,'T':19,'W':20,'Y':21,'V':22}
    with open(file, 'r') as inf:
        lines = inf.read().splitlines()
    pep_codes=[]
    long_pep_counter = 0
    newLines = []
    for pep in lines:
        if not len(pep) > max_len:
            current_pep=[]
            for aa in pep:
                current_pep.append(aa_dict[aa])
            pep_codes.append(current_pep)
            newLines.extend([pep])
        else:
            long_pep_counter += 1
    predict_data = keras.preprocessing.sequence.pad_sequences(pep_codes, value=0, padding='post', maxlen=max_len)
    return predict_data, long_pep_counter, newLines






In [43]:
def convertClass(stri):
    if stri == 'MObs':
        return 1
    elif stri == 'LObs':
        return 0
    else:
        return -1


def read_data_display_stat(ifile):
    class_dict = {}
    with open (ifile, 'r') as f:
        read_data = [(column[0],convertClass(column[-1])) for column in csv.reader(f,delimiter='\t')]
    #print (first_row)
    read_data.pop(0)

    #calc statistics
    MObs_total = 0
    LObs_total = 0
    pep_lengths = []
    for element in read_data:
        if element[1]==1:
            MObs_total+=1
        if element[1]==0:
            LObs_total+=1
        pep_lengths.append(len(element[0]))
        class_dict[element[0]] = element[1]

    vals, counts = np.unique(pep_lengths, return_counts=True)

    #find mode
    mode_value = np.argwhere(counts == np.max(counts))
    #Display data statistics
    print()
    print(ifile)
    print("--------------------------------------------------------")
    print("Total Size : {0} {1}".format(len(read_data),"peptides"))
    print("Average Peptide Length : {}".format(np.mean(pep_lengths)))
    print("Maximum Peptide Length : {}".format(np.max(pep_lengths)))
    print("Minimum Peptide Length : {}".format(np.min(pep_lengths)))
    print("Median Peptide Length : {}".format(np.median(pep_lengths)))
    print("Mode Peptide Length : {}".format(mode_value[0][0]))
    print("Class Breakdown")
    print("MObs : {0} %".format((MObs_total/len(read_data))*100))
    print("LObs : {0} %".format((LObs_total/len(read_data))*100))
    print("--------------------------------------------------------")
    return read_data,class_dict

#Create model input file
def create_model_infile(input_fname,read_list):    
    with open(input_fname, 'w') as outfile:
        outfile.writelines(str(element[0]) + '\n' for element in read_list)





In [44]:
    
raw_data_dir_train = 'GPMDB_training_peptides.txt'
raw_data_dir_test = 'GPMDB_test_peptides.txt'
input_file_name = "test_input.txt"
read_data_display_stat(raw_data_dir_train)
datalist,truth_dict = read_data_display_stat(raw_data_dir_test)
create_model_infile(input_file_name,datalist)


GPMDB_training_peptides.txt
--------------------------------------------------------
Total Size : 74998 peptides
Average Peptide Length : 15.946931918184484
Maximum Peptide Length : 81
Minimum Peptide Length : 4
Median Peptide Length : 14.0
Mode Peptide Length : 7
Class Breakdown
MObs : 49.99866663111016 %
LObs : 50.00133336888983 %
--------------------------------------------------------

GPMDB_test_peptides.txt
--------------------------------------------------------
Total Size : 24997 peptides
Average Peptide Length : 15.970356442773133
Maximum Peptide Length : 79
Minimum Peptide Length : 4
Median Peptide Length : 14.0
Mode Peptide Length : 7
Class Breakdown
MObs : 49.993999279913595 %
LObs : 50.006000720086405 %
--------------------------------------------------------


In [45]:
predict_data, skipped,  lines = load_pep_and_codify(input_file_name, 81)
#print(predict_data)
print('Succesfully loaded {0} peptides and skipped {1}'.format(len(lines), str(skipped)))



Succesfully loaded 24997 peptides and skipped 0


## Model
The model includes the model definitation which usually is a class, model training, and other necessary parts.

- Model architecture: The input vector of the DeepMSPeptide classifier is created using a CNN in TensorFlow by assigning an integer to each amino acid. The input is standardized for the CNN by performing zero-padding because of the different lengths of the peptides which results in each vector with a size of 81 elements. The first layer of the model is an embedding layer. The second layer is a dropout layer to prevent over-fitting. The next layers include 1 or 2 1D convolutional layers applying 64 filters with a sliding window of 1 and a width of 3. Lastly, the model includes two additional hidden layers. Those layers include a single-unit layer with a sigmoid activation function and the loss function selected for training was the binary cross entropy. Each model is trained for 200 epochs in batches of 100 samples. 
- Training objectives: The model is pre trained and compiled
- Others: The model is pre trained and compiled
Code is partially adapted from https://github.com/vsegurar/DeepMSPeptide/blob/master/DeepMSPeptide/DeepMSPeptide.py


In [46]:
file_name = "out.txt"

print('Loading model...')
model_2_1D = keras.models.load_model('model_2_1D.h5')


print('Making predictions')
model_2_1D_pred = model_2_1D.predict(predict_data)
model_2_1D_pred = np.hstack((np.array(lines).reshape(len(lines), 1),model_2_1D_pred)).tolist()

Pred_output = []
for pred in model_2_1D_pred:
    if float(pred[1]) > 0.5:
        # pred.extend('0')
        Pred_output.append([pred[0], str(1-float(pred[1])), '0'])
    else:
        Pred_output.append([pred[0], str(1-float(pred[1])), '1'])
        # pred.extend('1')

outFile = '{0}_Predictions.txt'.format(file_name.split('.')[0])
print('Saving predictions to file {}'.format(outFile))
with open(outFile, 'w') as outf:
    outf.write('Peptide\tProb\tDetectability\n')
    outf.writelines('\t'.join(i) + '\n' for i in Pred_output)



Loading model...
Making predictions
Saving predictions to file out_Predictions.txt


# Results
Run cell below to show results. Measures include Confusion matrix values, Accuracy, Precision, Recall/Sensitivity, F1 Score, Specificity, and AUC


In [47]:
with open (outFile, 'r') as f:
        predictions = [(column[0],column[-1]) for column in csv.reader(f,delimiter='\t')]
        
predictions.pop(0)
labels = []
pred = [] 
ind = 0
tp_count = 0
tn_count = 0
fp_count = 0
fn_count = 0
for element in predictions:
    labels.append(truth_dict[element[0]]) 
    pred.append(int(element[1]))
    if truth_dict[element[0]] == int(element[1]):
        if int(element[1])==1:
            tp_count+=1
        else:
            tn_count+=1
    else:
        if int(element[1])==1:
            fp_count+=1
        else:
            fn_count+=1
      
#print(labels)
#print(pred)
print("--------------------------------------------")    
print("True Positive : {}".format(tp_count))
print("True Negative : {}".format(tn_count))
print("False Positive : {}".format(fp_count))
print("False Negative : {}".format(fn_count))

labels = np.array(labels)
pred = np.array(pred)

accuracy = (tp_count+tn_count)/(tp_count+tn_count+fp_count+fn_count)
precision = tp_count/(tp_count+fp_count)
recall = tp_count/(tp_count+fn_count)
f1_score = (2*precision*recall)/(precision+recall)
sensitivity = tn_count / (tn_count + fp_count)
auc = metrics.roc_auc_score(labels, pred)
print("--------------------------------------------")  
print("Accuracy : {}".format(accuracy))
print("Precision : {}".format(precision))
print("Recall/Specificity : {}".format(recall))
print("F1 Score : {}".format(f1_score))
print("Sensitivity : {}".format(sensitivity))
print("AUC : {}".format(auc))



--------------------------------------------
True Positive : 11097
True Negative : 8784
False Positive : 3716
False Negative : 1400
--------------------------------------------
Accuracy : 0.7953354402528303
Precision : 0.7491392695605211
Recall/Specificity : 0.8879731135472514
F1 Score : 0.8126693518857561
Sensitivity : 0.70272
AUC : 0.7953465567736256


## Model Comparison

This table was retrieved from the paper [1]

|Model|	AUC|	Accuracy|	Specificity|	Sensitivity|	F-score|
|---|---|---|---|---|---|
|This Project|0.7953|0.7953 |0.8880 | 0.7027|0.8126|
|1D-2C-CNN	|0.8570	|0.7953	|0.8880	|0.7027	|0.7744|
|1D-1C-CNN	|0.8568	|0.7917	|0.9097	|0.6737	|0.7638|
|RFa	|0.7549	|0.6924	|0.7746	|0.6103	|0.6649|
|SvmRa	|0.7384	|0.6813	|0.7830	|0.5797	|0.6453|
|DNNb	|0.7360	|0.6692	|0.6813	|0.6572	|0.6659|
|C5a	|0.7312	|0.6644	|0.6513	|0.6775	|0.6687|
|Nneta	|0.7148	|0.6723	|0.8329	|0.5118	|0.6097|
|Rparta	|0.6893	|0.6527	|0.7467	|0.5587	|0.6167|
|Nba	|0.6456	|0.5997	|0.7280	|0.4714	|0.5408|
|Plsa	|0.6350	|0.6043	|0.6396	|0.5690	|0.5898|
|Glma	|0.6349	|0.6036	|0.6426	|0.5646	|0.5875|
|GlmStepAICa	|0.6349	|0.6034	|0.6424	|0.5645	|0.5874|
|Gaussianc	|0.6342	|0.5983	|0.6121	|0.5845	|0.5927|
|Jripa	|0.6238	|0.6240	|0.6549	|0.5930	|0.6119|



# Discussion

   This project was able to partially reproduce the results of this paper. I was able to achieve the same accuracy, specificity, and sensitivity, however I got different results for AUC and F-score. The results I was able to achieve was still better than the other models that were being compared. Future work in this area could be testing the effects of extra layers on the effectiveness of the model to predict observabilty of the peptides. The easy part about reproducing this paper is that the model provided in the github was pre trained and pre compiled so that simplified things. The difficult part about reproducing the paper was that the paper didn't provide instructions on how to test the model. It told us how to use a python script that was written to take and input file and produce an output but did not specify details of the input format. I had to figure out on my own how to clean the test data to create the input file. The suggestions I will give to the author are to be more descriptive in the github read.ME. It would be nice to have clear instructions and even code on how to reproduce the results of the paper. 

# References

1. Serrano, G., Guruceaga, E., & Segura, V. (2020). DeepMSPeptide: peptide detectability prediction using deep learning. Bioinformatics, 36(4), 1279–1280. doi: 10.1093/bioinformatics/btz708. Advance Access Publication Date: September 14, 2019. Available online: https://academic.oup.com/bioinformatics/article/36/4/1279/5569881